In [12]:
import openai
import os, time

import dspy

from dotenv import load_dotenv
load_dotenv()

from AiVoice import start_assistant
from spotifyTest import get_random_saved_track, sp, play_track, search_for_song, change_volume

In [13]:
agent = dspy.LM("gpt-4", api_key=os.getenv("OPENAI_API_KEY"))  
lm_agent = dspy.configure(lm = agent)

In [14]:
print(agent("hello world!"))

['Hello! How can I assist you today?']


In [15]:
class SongModule(dspy.Signature):
    """given a preference, return an item that matches the preference, search for the best match. do not play a song if a match cannot be found. if given a playlist or album, play that instead."""
    song_preferences: str = dspy.InputField()

    item_title: str = dspy.OutputField()
    item_artist: str = dspy.OutputField()
    item_type: str = dspy.OutputField()  # "track", "album", or "playlist"

    
    item_uri: str = dspy.OutputField()
    

In [16]:
def get_random_saved_track_wrapper():
    track = get_random_saved_track(sp)
    return track

In [17]:
def play_playlist_or_album_wrapper(playlist_uri: str):
    play_playlist_or_album(sp, playlist_uri)

In [18]:
def search_for_song_wrapper(song_name: str):
    track = search_for_song(sp, song_name)
    return track

In [19]:
def play_song(song_uri):
    play_track(sp, song_uri)
    

In [20]:
song = dspy.ReAct(signature=SongModule, tools=[get_random_saved_track_wrapper, search_for_song_wrapper, ])

In [35]:
sc = song(song_preferences = "play something chill and lo-fi")
# print(sc)

volume_rate = 7
volume_incriment = 10

for i in range(volume_rate):
        change_volume(sp, -volume_incriment)


start_assistant(f'now playing the {sc.item_type} {sc.item_title} by {sc.item_artist}')

print(sc.item_uri)
play_track(sp, sc.item_uri)
time.sleep(0.5)



for i in range(volume_rate):
        change_volume(sp, volume_incriment)



spotify:playlist:20WL4ioR8GFNEU0LITMpEe


In [22]:
print(sc)

Prediction(
    trajectory={'thought_0': 'The user wants to listen to a Halloween playlist. I should search for a playlist that matches this description.', 'tool_name_0': 'search_for_song_wrapper', 'tool_args_0': {'song_name': 'halloween playlist'}, 'observation_0': {'songs': {'Goo Goo Muck': {'uri': 'spotify:track:3EEd6ldsPat620GVYMEhOP', 'artist': 'The Cramps'}, 'Halloween Holla Holla': {'uri': 'spotify:track:2OxvIuk4RmPmED9j2MYPv9', 'artist': "Gabby's Dollhouse"}, 'This Is Halloween': {'uri': 'spotify:track:3ZwLB2bhvpVp42V9NHJQoM', 'artist': 'Chantry Johnson'}}, 'albums': {'Halloween Motion Picture Soundtrack': {'uri': 'spotify:album:1eA3cq4WvDR4yVKChvZewP', 'artist': 'John Carpenter'}, 'Halloween Howls: Fun & Scary Music (Deluxe Edition)': {'uri': 'spotify:album:2kAB6U6XJQ9lXK7TPsmaR4', 'artist': 'Andrew Gold'}, 'Halloween Party Playlist': {'uri': 'spotify:album:1ZjgJeGfWlAls0ngGxjDU0', 'artist': 'Waya Spirit'}}, 'playlists': {'why are you so obsessed with halloween?': {'uri': 'spo